In [11]:
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import json 
import os
import pickle

def loadData():

    with open('quora_raw_train.json') as f:
        training_examples = json.load(f)
        
    with open('quora_raw_val.json') as f:
        val_examples = json.load(f)

    with open('quora_data_prepro.json') as f:
        vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
        vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created          Vocab_inv = {'the':1, 'sauce':2, .....}

    X_train_prelim = [training_examples[i]['question1'] for i in range(len(training_examples))] # Training examples before preprocessing
    Y_train_prelim = [training_examples[i]['question'] for i in range(len(training_examples))]

    X_val_prelim = [val_examples[i]['question1'] for i in range(len(val_examples))]
    Y_val_prelim = [val_examples[i]['question'] for i in range(len(val_examples))]

    X_train = []
    Y_train = []

    filename = "../glove.6B.100d.txt"

    lines = open(filename).readlines()
    print(len(lines))
    embeddings = {}
    a = 0
    for line in lines:
        a +=1
        print(a)
        word = line.split()[0]
        embedding = list(map(float, line.split()[1:]))
        if word in vocab.values():
            embeddings[int(vocab_inv[word])] = embedding # This is creating a dictionary with indexes corresponding to the position of the particular word in the vocabulary
            # embeddings = {1:embedding of 'the', 2: embedding of 'sauce'....}
        
    embedded_words = embeddings
    
# This is basically tokenising each sentence, and converting it into a list of word embeddings. X corresponds to a training example and Y corresponds to its paraphrase
    for i in range(len(X_train_prelim)): 
        print(i)
        sentence_x = X_train_prelim[i]
        sentence_y = Y_train_prelim[i]
        tokensx = word_tokenize(sentence_x)
        tokensy = word_tokenize(sentence_y)
        vectorx = []
        vectory = []
        for word in tokensx:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
#If the word is not in the embeddings given, then add a random vector (might have to do something about this :) ) (Potentially could write a spell checker to find misspellings)
                vectorx.append(-2*torch.rand(100,1)) #
        for word in tokensy:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
                vectory.append(-2*torch.rand(100,1))

        X_train.append(vectorx)
        Y_train.append(vectory)

    return embedded_words

In [14]:
from Decoder import Decoder
from Encoder import Encoder

input_size = 100
hidden_size_enc = 128
hidden_size_dec = 20
num_layers = 1
vocab_size = 5 #len(vocab)
dropout = 0.2

embedded_words = loadData()

encoder = Encoder(input_size,hidden_size_enc,dropout,vocab_size,num_layers, embedded_words)
decoder = Decoder(encoder, input_dims, hidden_size_dec)

2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871


KeyboardInterrupt: 